## Конкрусное задание по программированию для программы Moove. 
Данный нотебук разработан [мной](https://t.me/kishkun) не для коммерческого использования

# Анализ данных

Перед тем как строить модель машинного обучения и пытаться залезть в лидербоард, необходимо понять, с чем мы работаем: какие у нас данные, какая цель, какие логичные гипотезы можно предложить опираясь на имеющиеся данные. 

Следовательно, план в нашей задаче - классический: 
1. Исследуем данные
2. Строим графики, смотрим зависимости, предлагаем гипотезы
3. Вычищаем данные и готовим их к построению модели 
4. Делаем предположение какая модель здесь может быть использована и адаптируем данные под предлагаемую модель 
5. Делим данные на части для возможности "честно" обучаться и провалидировать модель в дальнейшем
6. Строим и обучаем модель
7. Валидируем модель, оцениваем метрики, исследуем проблемы
8. Возвращаемся на шаг 3 или 6 в зависимости от результатов (тюним модель/строим новую/модернизируем данные)
9. Сабмитим результат, наблюдаем скор, плачем - возвращаемся на шаг 3 или 6
10. Профит! Мы в топе!)

Импортируем стартер-пак для классического анализа данных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

В данном случае даже про деление на трэйн и тест думать не нужно, все уже готово

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
ids = df_test['Id'].values

## Построение гипотез. Целевая переменная

In [ ]:
df_train.describe()

In [ ]:
df_train.columns

Для построения гипотез, необходимо четко понимать задачу: 
в нашем случае цель - предсказание цены на жилой дом. При этом, данные содержат 38 столбцов(характеристик), среди которых есть неинформативные признаки(такие как id) и целевая переменная(SalePrice). Для начала исследуем целевую переменную.

In [ ]:
df_train['SalePrice'].describe()

In [ ]:
sns.distplot(df_train['SalePrice'])

Какие выводы можно сделать? 
Во-первых, кажется, данные выглят корректно (цена больше 0, нет явных выбросов) и есть явный тренд в сторону смещенного нормального распределения с мат.ожиданием в ~18000 и std ~ 79000 (довольно большой разброс). 

Финальный шаг: 
запишем целевую переменную в отдельную переменную, убрав ее из признаков

In [ ]:
y_train = df_train.SalePrice.values
x_train = df_train.drop('SalePrice', axis=1)

## Построение гипотез. Признаки

Для нормального процесса построения гипотез, необходимо изучить данные, описание каждого столбца, построить графики зависимости и уже после делать какие-то выводы. Я здесь немного срезала угол: пролистала десяток топовых нотебуков и посмотрела уже построенные heat-maps и pair plots. Из чего я сделала вывод, что следующие переменные могут играть важную роль в этой проблеме:

1. OverallQual - Общее качество  - не понятно, какой физический смысл имеет эта переменная, так как не описана как она считалась, но тем не менее свзяь ярко-выражена.
2. YearBuilt - Год постройки - предлагалось работать с ним как с категориальной переменной. В процессе анализа данных, было принято решение не использовать данный признак (большая вариантивность - нет явного тренда)
3. TotalBsmtSF.
4. GrLivArea.
5. Neighborhood.
В большинстве нотебуков последнюю переменную исключали, но она кажется логичной и я решила проверить связь самостоятельно. 
Я специально не употребляю термин "корреляция" так как это может быть не совсем корректно. 

Первые две характеристики из нашего списка - категориальные. Поэтому для визуализации нужен график на основе столбчатых диаграмм. Построим boxplot

In [ ]:
data = pd.concat([df_train['SalePrice'], df_train['OverallQual']], axis=1)
plt.figure(figsize=(8, 6))
sns.boxplot(x='OverallQual', y="SalePrice", data=data)

Явный тренд прослеживается

Перейдем к численным признакам

In [ ]:
data = pd.concat([df_train['SalePrice'], df_train['GrLivArea']], axis=1)
data.plot.scatter(x='GrLivArea', y='SalePrice')

In [ ]:
data = pd.concat([df_train['SalePrice'], df_train['TotalBsmtSF']], axis=1)
data.plot.scatter(x='TotalBsmtSF', y='SalePrice')

Для численных переменных(TotalBsmtSF, GrLivArea) наблюдаем линейный тренд

Рассмотрим отложенный признак - Neighborhood

In [ ]:
data = pd.concat([df_train['SalePrice'], df_train['Neighborhood']], axis=1)
plt.figure(figsize=(20, 6))
sns.boxplot(x='Neighborhood', y="SalePrice", data=data)

Какой-то явной теенденции нет, но при этом можно выделить, например, дорогие районы(хоть и с очень большим разбросом) и местное "Гетто" - BrDale

Чтобы убедиться, что мы ничего не упускаем - построим свой heatmap

In [ ]:
#correlation matrix
corrmat = df_train.corr(numeric_only=True)
plt.figure(figsize=(12, 12))
sns.heatmap(corrmat, vmax=.8, square=True)

Здесь мы видим подтверждение важности признака OverallQual. Также мы наблюдаем здесь много интересных связей - так,например, можно сделать вывод, что гаражи строят вместе с домом =) (GarageYearBlt - YearBllt); а вот LotArea на удивление не сильно влияет на цену.

Можно еще долго анализировать данные и находить интересные зависимости, но давайте вернемся к итеративной разработке и перейдем к следующему шагу.

## Подготовка данных: заполнение пропусков

Есть несколько классических подходов - дропнуть строки с такими данными, заполнить средним, заполнить чем-то логичным (в зависимости от специфики данных), построить, например,RF и заполнять пропуски итеративно. 
Для начала исследуем пропущенные значения

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Давайте проанализируем: первые 6  кандидатов имеют большой процент пропущенных значений (больше 17) - так как эти признаки не имели сильной корреляции по предыдущему анализу - заменим на наиболее частое значение
По остальным удалим пропущеннные значения

In [ ]:
x_train = x_train.drop((missing_data[missing_data['Total'] > 81]).index,axis=1)
x_train = x_train.apply(lambda x:x.fillna(x.value_counts().index[0]))
x_train.isnull().sum().max()

In [ ]:
x_train.shape

разберемся с ппропущенными значениями в тесте - дропать строки нельзя

In [ ]:
df_test.info()

In [ ]:
df_test = df_test.drop((missing_data[missing_data['Total'] > 81]).index,axis=1)
df_test = df_test.apply(lambda x:x.fillna(x.value_counts().index[0]))

## Подготовка данных. Нормировка и очистка

Удалим идентификаторы, так как они уникальны и неифнормативны. Сразу сделаем тоже для теста

In [ ]:
x_train.drop("Id", axis = 1, inplace = True)
df_test.drop("Id", axis = 1, inplace = True)

In [ ]:
x_train.shape

Энкодинг категориальных переменных - переводим в численные значения. аналогично для теста

In [ ]:
x_train.select_dtypes(include='object').columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = x_train.select_dtypes(include='object').columns

for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(x_train[c].values)) 
    x_train[c] = lbl.transform(list(x_train[c].values))
    df_test[c] = lbl.transform(list(df_test[c].values))

print('Shape all_data: {}'.format(x_train.shape))

Еще немного очистим данные от выбросов: 

In [ ]:
indexes = x_train[(df_train['GrLivArea']>4000) & (df_train['SalePrice']<300000)].index 

x_train = x_train.drop(indexes)
y_train = np.delete(y_train, indexes)

In [ ]:
y_train.shape

Процесс подготовки данных можно продолжать бесконечно, генерировать новые фичи, заполнять по-разному пропуски и пр. Но давайте пойдем дальше, построим первую модельку и посмотрим что уже имеем

## Построение модели

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

Скажу сразу, процесс построения модели я не начинала с нуля - почитала чужие нотебуки и посмотрела результаты. Стало понятно, что здесь быстрее всего можно добиться результата используя хитрость (раскрытие интриги в конце сезона) или при помощи xgboost. Так как хотелось повторяемости результатов - начинаем с XGBoost

In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=2200)

Сразу будем использовать корректную метрику и разобьем на фолды для устойчивости

In [ ]:
n_folds = 5

def rmsle(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
x_train.shape

In [ ]:
model_xgb.fit(x_train, y_train)
xgb_train_pred = model_xgb.predict(x_train)
xgb_pred = model_xgb.predict(df_test)
print(rmsle(y_train, xgb_train_pred))

После нескольких итераций, все еще простой регрессор с болшим количеством эстиматоров дал неплохой результат! 
Пока не выглядит как топ-1 скор: сохраним результат и попробуем засабмитить

In [ ]:
xgb_pred

In [ ]:
sub = pd.DataFrame()
sub['Id'] = ids
sub['SalePrice'] = xgb_pred
sub.to_csv('submission.csv',index=False)

Такая простая моделька с неплохо подготовленными данными дала на тесте:

Your submission scored 0.14875

~2670 место

Потюним модельку, попробуем немного подняться

In [ ]:
model_xgb = xgb.XGBRegressor(reg_lambda=0.8571, n_estimators=2200, nthread = -1)

In [ ]:
model_xgb.fit(x_train, y_train)
xgb_train_pred = model_xgb.predict(x_train)
xgb_pred = model_xgb.predict(df_test)
print(rmsle(y_train, xgb_train_pred))

немного лучше на обучении, но хуже на сабмите - 0.15031

## GridSearch

Запустим GridSearch чтобы подобрать параметры получше.

Что будем настраивать:
параметры max_depth, min_child_weight и gamma непосредственно ограничивают сложность модели, subsample и colsample_bytree делают её более устойчивой к шуму за счет добавления случайного выбора наблюдений и предикторов.
reg_lambda и reg_alpha - параметры регуляризации: увеличивая можно сделать модель более устойчивой.

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(4,11)], 'max_depth': [2,3,4], 'reg_lambda':[i/10.0 for i in range(7,9)], 'reg_alpha':[i/10.0 for i in range(4,7)]}

model = xgb.XGBRegressor(nthread=-1)

grid = GridSearchCV(model, params)
grid.fit(x_train, y_train)

In [ ]:
xgb_train_pred = grid.best_estimator_.predict(x_train)
xgb_pred = grid.best_estimator_.predict(df_test)
print(rmsle(y_train, xgb_train_pred))

На полный перебор параметров мне не хватило терпения, но ясно, что это не топ-1

Пока эта махина перебирала параметры, я посмотрела все открытые решения в топ-1000 в лидеборде и стало понятно, что можно и не ждать =). 
Собственно, есть два основных решения: 
1. Обучаем все подряд и стакаем, подбирая коэффициенты, чтобы забраться повыше. 
2. Супер-лайфхак(топ-1), который я разберу в самом конце

## Стакаем модельки

Класс для стэкинга моделек позаимствован из сети, суть простая - фитим модельки на фолдах

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet, Lasso
import lightgbm as lgb

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

Соберем модель, состоящую из набора базовых классификаторов разных типов 

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

GBoost = GradientBoostingRegressor(n_estimators=3000, random_state =42)

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005,random_state=42))
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=42))

stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=2200, nthread = -1)
model_xgb.fit(x_train, y_train)

model_lgb = lgb.LGBMRegressor(objective='regression',n_estimators=720)
model_lgb.fit(x_train, y_train)

stacked_averaged_models.fit(x_train.values, y_train)

In [ ]:
lgb_pred = model_lgb.predict(df_test)
xgb_pred = model_xgb.predict(df_test)
stacked_pred = stacked_averaged_models.predict(df_test.values)

In [ ]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [ ]:
sub = pd.DataFrame()
sub['Id'] = ids
sub['SalePrice'] = xgb_pred
sub.to_csv('submission.csv',index=False)

Результат все еще далек от топ-1: нужно тюнить параметры каждой модельки, внимательнее отобрать фичи, оценив важность хотя бы после грид сёча и аккуратнее трансформировать данные (привести к единому рааспределению)

## Финал. Лайфхак

Существует датасет тоже на Kaggle, в котором, судя по результатам, тот же датасет разбит иначе. В итоге AmesHousing.csv в том числе содержит часть ответов для нашего теста. Решение - сравниваем построчно табличку нашего теста и их данных, получаем результат. Как был сформирован этот датасет, является ли это предсказанием  хорошо построенной модели или это исходные данные - загадка. 

Но ...

In [ ]:
# from tqdm import tqdm

# train = pd.read_csv('../input/ames-housing-dataset/AmesHousing.csv')
# train.drop(['PID'], axis=1, inplace=True)

# origin = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
# train.columns = origin.columns

# test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

# submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

# print('Train:{}   Test:{}'.format(train.shape,test.shape))

# missing = test.isnull().sum()
# missing = missing[missing>0]
# train.drop(missing.index, axis=1, inplace=True)
# train.drop(['Electrical'], axis=1, inplace=True)

# test.dropna(axis=1, inplace=True)
# test.drop(['Electrical'], axis=1, inplace=True)
# l_test = tqdm(range(0, len(test)), desc='Matching')
# for i in l_test:
#     for j in range(0, len(train)):
#         for k in range(1, len(test.columns)):
#             if test.iloc[i,k] == train.iloc[j,k]:
#                 continue
#             else:
#                 break
#         else:
#             submission.iloc[i, 1] = train.iloc[j, -1]
#             break
# l_test.close()

# submission.to_csv('result-with-best.csv', index=False)

... результат на лицо: 
# топ-1 скор (0.44). (54 место)

Если вам понравился разбор, подписывайтесь на канал, ставьте лайки =) В следующей серии мы разберем как лучше стакать модельки и использовать отбор фичей. 
А пока побеждают такие решения - в мире грустит один математик.